In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('/Users/ZHAOFENG SHANG/Desktop/NoteBook/PriceSuggestion/train_small2.tsv', encoding='utf-8')

In [3]:
train.head()

,Unnamed: 0,name,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,NaN,0
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,14
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,10
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,14
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,3


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 9 columns):
Unnamed: 0           25000 non-null int64
name                 25000 non-null object
item_condition_id    25000 non-null int64
category_name        24880 non-null object
brand_name           14257 non-null object
price                25000 non-null float64
shipping             25000 non-null int64
item_description     23608 non-null object
desc_len             25000 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 1.3+ MB


In [5]:
def text_process(mess):

    nonpunc = [char for char in mess if char not in string.punctuation]# in a string for every words

    nonpunc = ''.join(nonpunc)
    # first need split the nonpunc('string'),then use lower case word to check stopwords
    return [words for words in nonpunc.split() if words.lower() not in stopwords.words('english')]

In [6]:
train.loc[train['item_description']=='No description yet','item_description']='NULL'

In [7]:
train.loc[train['item_description'].isnull(), 'item_description'] = 'NULL'

In [8]:
X = (train['name'] + ' ' + train['item_description']).values
Y = np.log1p(train['price'].values)

In [9]:
X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size=0.3)

In [10]:
import string
from nltk.corpus import stopwords
vect = TfidfVectorizer(analyzer=text_process)
X_train_vect = vect.fit_transform(X_train)


In [20]:
X_test_vect = vect.fit_transform(X_test)


In [21]:
y_train

array([ 3.29583687,  2.39789527,  1.60943791, ...,  2.7080502 ,
        2.30258509,  2.30258509])

In [22]:
model = SGDRegressor(loss='squared_loss', penalty='l2', max_iter=5)
params = {'penalty':['none','l2','l1'],
          'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}
gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)

In [23]:
gs.fit(X_train_vect, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1.9837360101274717, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1.8926709616344966, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1.8589917682572523, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-2.003186081155002, total=   0.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-1.8678049120127937, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1.9871000653619595, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ...................................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  alpha=0.0001, penalty=l2, score=-1.8957955151987085, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1.8624950799653008, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-2.0073648103984922, total=   0.0s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-1.8704828339772466, total=   0.0s
[CV] alpha=0.0001, penalty=l1 ........................................
[CV]  alpha=0.0001, penalty=l1, score=-2.0098754615685825, total=   0.0s
[CV] alpha=0.0001, penalty=l1 ........................................
[CV]  alpha=0.0001, penalty=l1, score=-1.9160610474953106, total=   0.0s
[CV] alpha=0.0001, penalty=l1 ........................................
[CV]  alpha=0.0001, penalty=l1, score=-1.8840691389916238, total=   0.0s
[CV] alpha=0.0001, penalty=l1 .................................

[CV]  alpha=0.002, penalty=l1, score=-2.311029844366723, total=   0.0s
[CV] alpha=0.002, penalty=l1 .........................................
[CV]  alpha=0.002, penalty=l1, score=-2.2051718218952425, total=   0.0s
[CV] alpha=0.002, penalty=l1 .........................................
[CV]  alpha=0.002, penalty=l1, score=-2.179148004984871, total=   0.0s
[CV] alpha=0.002, penalty=l1 .........................................
[CV]  alpha=0.002, penalty=l1, score=-2.328405894934342, total=   0.0s
[CV] alpha=0.002, penalty=l1 .........................................
[CV]  alpha=0.002, penalty=l1, score=-2.184958178482023, total=   0.0s
[CV] alpha=0.005, penalty=none .......................................
[CV]  alpha=0.005, penalty=none, score=-1.9846347219447684, total=   0.0s
[CV] alpha=0.005, penalty=none .......................................
[CV]  alpha=0.005, penalty=none, score=-1.8925857225078606, total=   0.0s
[CV] alpha=0.005, penalty=none .......................................

[CV]  alpha=0.05, penalty=l2, score=-2.9908890059295716, total=   0.0s
[CV] alpha=0.05, penalty=l2 ..........................................
[CV] . alpha=0.05, penalty=l2, score=-2.843009297416894, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[CV]  alpha=0.05, penalty=l1, score=-3.4517796583054383, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[CV] . alpha=0.05, penalty=l1, score=-3.346464949912977, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[CV]  alpha=0.05, penalty=l1, score=-3.3048573992962598, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[CV]  alpha=0.05, penalty=l1, score=-3.4759318807780266, total=   0.0s
[CV] alpha=0.05, penalty=l1 ..........................................
[CV]  alpha=0.05, penalty=l1, score=-3.3322516363579924, total=   0.0s
[CV] alpha=0.1, penalty=none .........................................
[CV]  

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    4.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['none', 'l2', 'l1'], 'alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [24]:
print("Best score: %0.3f" % gs.best_score_)
print("Best parameters set:")
bsp = gs.best_estimator_.get_params()  # the dict of parameters with best score
    
for param_name in sorted(bsp.keys()):
    print("\t%s: %s" % (param_name, bsp[param_name]))

Best score: -1.921
Best parameters set:
	alpha: 0.01
	average: False
	epsilon: 0.1
	eta0: 0.01
	fit_intercept: True
	l1_ratio: 0.15
	learning_rate: invscaling
	loss: squared_loss
	max_iter: 5
	n_iter: None
	penalty: none
	power_t: 0.25
	random_state: None
	shuffle: True
	tol: None
	verbose: 0
	warm_start: False


In [25]:
print(gs.best_params_)

{'alpha': 0.01, 'penalty': 'none'}


In [26]:
model = SGDRegressor(**bsp)

In [27]:
model.fit(X_train_vect, y_train)


SGDRegressor(alpha=0.01, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='none',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [28]:
Y_predict = model.predict(X_test_vect)

ValueError: dimension mismatch

In [62]:
import re
line = "President [P] Barack Obama [/P] met Microsoft founder [P] Bill Gates [/P], yesterday."
re.findall('\[P\]\s?(.+?)\s?\[\/P\]', line)


['Barack Obama', 'Bill Gates']

In [29]:
X_test_vect

<7500x24028 sparse matrix of type '<class 'numpy.float64'>'
	with 143120 stored elements in Compressed Sparse Row format>

In [30]:
X_train_vect

<17500x38964 sparse matrix of type '<class 'numpy.float64'>'
	with 333831 stored elements in Compressed Sparse Row format>